# 11. Position Sizing and Transaction Costs (Manual Trading Research)

> ⚠️ Research tutorial only. Not financial advice.

このノートでは、手動売買で使えるサイズ設計を2種類比較します。

1. **Fixed 1 Unit**（常に同じエクスポージャ）
2. **Volatility Targeting (ATR)**（ATRが高いほどサイズを下げる）

さらに、簡易コストモデル（手数料 + スリッページ）を加え、**グロス成績とネット成績の差**を確認します。

## 1) コンセプトと数式\n\n日次リターンを $r_t$、ポジションウェイトを $w_t$ とすると、\nリーケージ回避のため以下を使います。\n\n$$\nr^{gross}_t = w_{t-1} \cdot r_t\n$$\n\nコストはウェイト変化量に比例させます。\n\n$$\ncost_t = \frac{(fee_{bps}+slip_{bps})\cdot |w_t-w_{t-1}|}{10000}\n$$\n\n$$\nr^{net}_t = r^{gross}_t - cost_t\n$$\n\nATRベースのボラティリティ・ターゲットは概念的に、\n\n$$\nw_t \propto \frac{\sigma^*}{ATR_t / Close_t}\n$$\n\nとして計算し、上限レバレッジでクリップします。

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ノートブック実行位置が root / notebooks どちらでも import できるように調整
repo_root = Path.cwd()
if not (repo_root / 'src').exists():
    repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root / 'src'))

from quantlab.data import fetch_ohlc
from quantlab.indicators import atr, ema

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
plt.style.use('seaborn-v0_8')

In [ ]:
# --- 実験パラメータ（必要に応じて調整） ---
SYMBOL = 'SPY'
PERIOD = '5y'
INTERVAL = '1d'

EMA_FAST = 12
EMA_SLOW = 26
ATR_N = 14

TARGET_DAILY_VOL = 0.01
MAX_LEVERAGE = 1.5
FEE_BPS = 2.0
SLIPPAGE_BPS = 3.0

try:
    price = fetch_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL)
except Exception as e:
    print(f'Fetch failed ({e}); using synthetic data.')
    idx = pd.date_range('2020-01-01', periods=900, freq='B')
    rnd = np.random.default_rng(42).normal(0.0003, 0.012, size=len(idx))
    close = 100 * (1 + pd.Series(rnd, index=idx)).cumprod()
    price = pd.DataFrame(index=idx)
    price['Close'] = close
    price['Open'] = price['Close'].shift(1).fillna(price['Close'])
    price['High'] = np.maximum(price['Open'], price['Close']) * 1.003
    price['Low'] = np.minimum(price['Open'], price['Close']) * 0.997
    price['Volume'] = 1_000_000

price.head()

## 2) シグナルとサイズ計算

In [ ]:
df = price.copy()

# 方向シグナル: EMA fast が slow を上回れば 1（ロング）、それ以外 0
df['ema_fast'] = ema(df['Close'], EMA_FAST)
df['ema_slow'] = ema(df['Close'], EMA_SLOW)
df['signal'] = (df['ema_fast'] > df['ema_slow']).astype(float)

# ATR 比率（価格で割る）をボラの近似として利用
df['atr'] = atr(df, ATR_N)
df['atr_ratio'] = (df['atr'] / df['Close']).replace(0, np.nan)

# 1) 固定サイズ: signal のとき常に1.0
df['w_fixed'] = df['signal']

# 2) ボラターゲット: 目標ボラ / 推定ボラ
vol_scaled = TARGET_DAILY_VOL / df['atr_ratio']
df['w_vol_target'] = (df['signal'] * vol_scaled).clip(lower=0.0, upper=MAX_LEVERAGE)
df[['Close', 'signal', 'atr_ratio', 'w_fixed', 'w_vol_target']].tail()

## 3) コスト込みリターン計算（グロス vs ネット）

In [ ]:
def backtest_with_costs(close: pd.Series, weight: pd.Series, fee_bps: float, slippage_bps: float) -> pd.DataFrame:
    # close と target weight から、グロス/ネット日次リターンを計算する
    # 教材目的のため、コストは |Δweight| 比例の単純モデルを採用
    out = pd.DataFrame(index=close.index)
    out['ret'] = close.pct_change().fillna(0.0)
    out['w'] = weight.fillna(0.0)
    out['dw'] = out['w'].diff().abs().fillna(0.0)

    # シグナル当日ではなく翌日に効かせる（look-ahead 回避）
    out['gross_ret'] = out['w'].shift(1).fillna(0.0) * out['ret']

    # コスト = (fee + slippage) * turnover 的な近似
    total_bps = fee_bps + slippage_bps
    out['cost'] = (total_bps / 10_000.0) * out['dw']
    out['net_ret'] = out['gross_ret'] - out['cost']

    out['eq_gross'] = (1 + out['gross_ret']).cumprod()
    out['eq_net'] = (1 + out['net_ret']).cumprod()
    return out

bt_fixed = backtest_with_costs(df['Close'], df['w_fixed'], FEE_BPS, SLIPPAGE_BPS)
bt_vt = backtest_with_costs(df['Close'], df['w_vol_target'], FEE_BPS, SLIPPAGE_BPS)

In [ ]:
def summarize(bt: pd.DataFrame) -> pd.Series:
    ann = 252
    ann_ret = (1 + bt['net_ret']).prod() ** (ann / max(len(bt), 1)) - 1
    ann_vol = bt['net_ret'].std() * np.sqrt(ann)
    cum = bt['eq_net']
    dd = cum / cum.cummax() - 1
    turnover = bt['dw'].sum()
    return pd.Series({
        'Total Return (net)': cum.iloc[-1] - 1,
        'Annual Return (net)': ann_ret,
        'Annual Vol (net)': ann_vol,
        'Max Drawdown (net)': dd.min(),
        'Turnover Sum': turnover,
        'Cost Drag (gross-net)': bt['gross_ret'].sum() - bt['net_ret'].sum(),
    })

summary = pd.DataFrame({
    'Fixed 1 Unit': summarize(bt_fixed),
    'Vol Target (ATR)': summarize(bt_vt),
})
summary

### 数値例（直近5日）

In [ ]:
example = pd.DataFrame({
    'Close': df['Close'],
    'w_fixed': bt_fixed['w'],
    'w_vt': bt_vt['w'],
    'gross_ret_fixed': bt_fixed['gross_ret'],
    'net_ret_fixed': bt_fixed['net_ret'],
    'gross_ret_vt': bt_vt['gross_ret'],
    'net_ret_vt': bt_vt['net_ret'],
}).tail(5)
print(example.to_string())

## 4) 可視化

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# 累積リターン（グロス/ネット）比較
axes[0].plot(bt_fixed.index, bt_fixed['eq_gross'], label='Fixed Gross', alpha=0.6)
axes[0].plot(bt_fixed.index, bt_fixed['eq_net'], label='Fixed Net', linewidth=2)
axes[0].plot(bt_vt.index, bt_vt['eq_gross'], label='VolTarget Gross', alpha=0.6)
axes[0].plot(bt_vt.index, bt_vt['eq_net'], label='VolTarget Net', linewidth=2)
axes[0].set_title(f'{SYMBOL}: Equity Curves (Gross vs Net)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ポジションサイズ推移
axes[1].plot(df.index, df['w_fixed'], label='w_fixed', alpha=0.8)
axes[1].plot(df.index, df['w_vol_target'], label='w_vol_target', alpha=0.8)
axes[1].set_title('Position Weights')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5) What to observe（観察ポイント）

- Vol Target は荒い局面でサイズが落ちるため、固定サイズよりDDが緩和される場合があります。
- 一方で turnover が増えると、コストで優位性が削られます。
- **グロスでは良いのにネットで悪化**するなら、売買頻度の抑制やシグナル安定化が次の改善候補です。